In [5]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG19
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [2]:
# from tensorflow import keras

# keras.backend.clear_session()

In [1]:
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

train_dir= r'Train'
test_dir= r'Test'
validation_dir= r'Validation'

# Load ResNet50 pre-trained model
conv_base = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Fine-tune the last few layers
conv_base.trainable = True
for layer in conv_base.layers[:-5]:
    layer.trainable = False

# Build the model with additional dense layers
model = models.Sequential([
    conv_base,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'), 
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.Adam(learning_rate=1e-3),
              metrics=['acc'])

# Data augmentation 
train_datagen = ImageDataGenerator(
    rescale=1./255,              
    rotation_range=20,           
    width_shift_range=0.1,       
    height_shift_range=0.1,      
    shear_range=0.1,             
    zoom_range=0.1,              
    horizontal_flip=True,        
    fill_mode='nearest'          
)

# Rescaling for validation and test sets
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Load the datasets
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

# Calculate steps per epoch
steps_per_epoch = train_generator.samples // train_generator.batch_size
validation_steps = validation_generator.samples // validation_generator.batch_size

# Callbacks
checkpoint_cb = ModelCheckpoint("best_model_resnet50.h5", save_best_only=True, monitor="val_loss", mode="min")
early_stopping_cb = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model with early stopping
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=30,  # Increased epochs for better training
    validation_data=validation_generator,
    validation_steps=validation_steps,
    callbacks=[checkpoint_cb, early_stopping_cb]
)

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_generator)
print(f'Test accuracy: {test_acc}')


C:\Users\hosse\anaconda3\envs\geospatial\lib\site-packages\h5py\__init__.py:36: UserWarning: h5py is running against HDF5 1.12.2 when it was built against 1.12.1, this may cause problems
  _warn(("h5py is running against HDF5 {0} when it was built against {1}, "


Found 10000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.
Found 992 images belonging to 2 classes.
Epoch 1/30
312/312 [==============================] - 333s 1s/step - loss: 0.5259 - acc: 0.7371 - val_loss: 0.8524 - val_acc: 0.5725
Epoch 2/30
312/312 [==============================] - 331s 1s/step - loss: 0.4031 - acc: 0.8210 - val_loss: 0.6635 - val_acc: 0.6787
Epoch 3/30
312/312 [==============================] - 331s 1s/step - loss: 0.3490 - acc: 0.8512 - val_loss: 0.3124 - val_acc: 0.8487
Epoch 4/30
312/312 [==============================] - 329s 1s/step - loss: 0.3009 - acc: 0.8779 - val_loss: 0.2602 - val_acc: 0.8813
Epoch 5/30
312/312 [==============================] - 330s 1s/step - loss: 0.2929 - acc: 0.8811 - val_loss: 0.2195 - val_acc: 0.9262
Epoch 6/30
312/312 [==============================] - 345s 1s/step - loss: 0.2777 - acc: 0.8857 - val_loss: 0.2202 - val_acc: 0.8950
Epoch 7/30
312/312 [==============================] - 331s 1s/step - loss: 0.

Real Time Mask Detection

In [6]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Load the pre-trained model
model = load_model("final_resnet50_model.h5")

# Define Haar Cascade for face detection
cascade_path = r'C:\Users\hosse\Downloads\Final_Assignment_Face Mask Dataset\haarcascade_frontalface_default.xml'
face_clsfr = cv2.CascadeClassifier(cascade_path)

# Labels and colors for mask detection
labels_dict = {0: 'with_mask', 1: 'without_mask'}
color_dict = {0: (0, 255, 0), 1: (0, 0, 255)}  

# Initialize webcam
webcam = cv2.VideoCapture(0) 

# Ensure webcam opened successfully
if not webcam.isOpened():
    print("Error: Could not open webcam.")
    exit()

size = 4  # Resize scale factor for speeding up detection

while True:
    rval, im = webcam.read() 
    if not rval:
        print("Error: Could not read from webcam.")
        break

    im = cv2.flip(im, 1)  # Flip horizontally (like a mirror)

    # Resize image to speed up face detection
    mini = cv2.resize(im, (im.shape[1] // size, im.shape[0] // size))

    # Detect faces in the resized image
    faces = face_clsfr.detectMultiScale(mini, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Loop through detected faces
    for (x, y, w, h) in faces:
        (x, y, w, h) = [v * size for v in (x, y, w, h)]  # Scale back to original size
        face_img = im[y:y+h, x:x+w]  # Crop the face from the image

        # Preprocess the face image
        resized = cv2.resize(face_img, (224, 224))  # Resize to (224, 224) as required by the model
        normalized = resized / 255.0  # Normalize the pixel values to [0, 1]
        reshaped = np.reshape(normalized, (1, 224, 224, 3))  # Reshape to match model input (1, 224, 224, 3)

        cv2.imshow("Cropped Face", face_img)
        cv2.waitKey(1)  
        
        # Predict mask status
        result = model.predict(reshaped)
        probability = result[0][0]  # Get the sigmoid output probability

        # Set a threshold for binary classification (default is 0.5)
        if probability > 0.5:
            label = 1  # "without_mask"
        else:
            label = 0  # "with_mask"

        # Draw a rectangle around the face and display the label
        cv2.rectangle(im, (x, y), (x+w, y+h), color_dict[label], 2)
        cv2.rectangle(im, (x, y-40), (x+w, y), color_dict[label], -1)  # Draw filled rectangle for label background
        cv2.putText(im, labels_dict[label], (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

    # Display the live image with detections
    cv2.imshow('LIVE', im)

    # Exit the loop on 'Esc' key
    key = cv2.waitKey(10)
    if key == 27:  # 27 is the ASCII code for the 'Esc' key
        break

# Release the webcam and close all windows
webcam.release()
cv2.destroyAllWindows()
